In [6]:
%pip install datasets python-Levenshtein -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd

splits = {
    "train": "train.jsonl",
    "validation": "validation.jsonl",
    "test": "test.jsonl",
}

df = pd.read_json(
    "hf://datasets/ai-forever/kinopoisk-sentiment-classification/" + splits["train"],
    lines=True,
)

texts = df.text.to_list()

In [2]:
texts[0]

'Если честно, меня не очень впечатлила новость, о том, что Гай Ричи собирается снять фильм, о Шерлоке Холмсе. Подумал — да это же будет: Карты, деньги, два ствола — у Холмса и у Ватсона. Но затем по мере появления трейлеров, и большей информации поменял своё отношение.\n«Шерлок Холмс» — последняя картина, на которую я планировал пойти в этом году. Жутко боялся, что она меня разочарует т. к. перед этим были расстроившие «Безумный спецназ», «Так себе каникулы» и немного 2012. Но Холмс полностью оправдал доверия.\nСюжет\nВ ленте динамичный, а главное интересный и захватывающий сюжет, что в последнее время не так уж и часто. Новый Холмс не поход классический образ представленный в картинах Игоря Масленникова, но это не портит его образ. Он больше подобен на Тони Старку из «Железного человека» или Грегори Хаусу из сериала «Доктор Хаус». Как и они, он весьма харизматичен, слегка безумен, но гениален в своём любимом деле.\nНа первый взгляд сюжет портит сверхъестественное восстание из могилы л

Составляем словарь букв

In [3]:
import re
import string

# Составляем словарь букв
vocab = set()
punctuation = string.punctuation

for text in texts:
    text = re.sub(f"[{punctuation}]", "", text)
    text = re.sub(r"[\t\n ]", "", text)
    text = text.lower()
    vocab.update(text)

# Выводим всё небуквенное
sorted(
    vocab - 
    set(
        string.ascii_lowercase + 
        string.ascii_uppercase + 
        string.digits + 
        "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
    )
)

['\\', '©', '«', '®', '±', '·', '»', 'і', '—', '‘', '’', '“', '„', '•', '…']

In [4]:
# Словарь, основанный на whitelist'е
vocab = set(
    string.ascii_lowercase +
    string.digits + 
    "абвгдеёжзийклмнопрстуфхцчшщъыьэюя"
)

Распределяем точки равномерно по сфере

In [45]:
import importlib
import calculate_distances
importlib.reload(calculate_distances)
from calculate_distances import calculate_distances, optimize_coulomb_energy

# Распределяем точки равномерно по сфере
dim = 24
points, energy = optimize_coulomb_energy(
    N=len(vocab), d=dim, lr=5e-3, n_iter=96, log_iter=16
)

# Считаем расстояния после обучения
calculate_distances(points)

Iter   16 | Energy = 28472.8516
Iter   32 | Energy = 28421.9160
Iter   48 | Energy = 28394.1387
Iter   64 | Energy = 28375.4570
Iter   80 | Energy = 28362.3848
Iter   96 | Energy = 28352.9785
Минимальное косинусное расстояние: 0.7425143995549892
Среднее косинусное расстояние: 0.9999999935175443
Дисперсия косинусного расстояния: 0.04176415857335563
Максимальное косинусное расстояние: 1.849953337523781

Минимальное евклидово расстояние: 1.2186175422534782
Среднее евклидово расстояние: 1.3997493324612353
Дисперсия евклидово расстояния: 0.0407017866448335
Максимальное евклидово расстояние: 1.923514066909535


In [46]:
# Делаем match векторов и словаря
vocab = sorted(vocab)
vocab_dict = {char: point for char, point in zip(vocab, points)}

# Сохраняем словарь в файл
import pickle

with open("vocab_repelled_vectors.pkl", "wb") as f:
    pickle.dump(vocab_dict, f)

# Загружаем словарь из файла
with open("vocab_repelled_vectors.pkl", "rb") as f:
    vocab_dict = pickle.load(f)

In [47]:
# Убираем из текста всё что не входит в словарь
vocab_str = ''.join(vocab)
text = re.sub(f"[^{vocab_str} ]", "", texts[0].lower())

# Разбиваем текст на слова
words = text.split()

# Разбиваем слова на n-граммы
n_grams_len = 3
ngrams = [[word[i:i+n_grams_len] for i in range(max(1, len(word) - n_grams_len + 1, 1))] for word in words]

ngrams

[['есл', 'сли'],
 ['чес', 'ест', 'стн', 'тно'],
 ['мен', 'еня'],
 ['не'],
 ['оче', 'чен', 'ень'],
 ['впе', 'печ', 'еча', 'чат', 'атл', 'тли', 'лил', 'ила'],
 ['нов', 'ово', 'вос', 'ост', 'сть'],
 ['о'],
 ['том'],
 ['что'],
 ['гай'],
 ['рич', 'ичи'],
 ['соб', 'оби', 'бир', 'ира', 'рае', 'ает', 'етс', 'тся'],
 ['сня', 'нят', 'ять'],
 ['фил', 'иль', 'льм'],
 ['о'],
 ['шер', 'ерл', 'рло', 'лок', 'оке'],
 ['хол', 'олм', 'лмс', 'мсе'],
 ['под', 'оду', 'дум', 'ума', 'мал'],
 ['да'],
 ['это'],
 ['же'],
 ['буд', 'уде', 'дет'],
 ['кар', 'арт', 'рты'],
 ['ден', 'ень', 'ньг', 'ьги'],
 ['два'],
 ['ств', 'тво', 'вол', 'ола'],
 ['у'],
 ['хол', 'олм', 'лмс', 'мса'],
 ['и'],
 ['у'],
 ['ват', 'атс', 'тсо', 'сон', 'она'],
 ['но'],
 ['зат', 'ате', 'тем'],
 ['по'],
 ['мер', 'ере'],
 ['поя', 'ояв', 'явл', 'вле', 'лен', 'ени', 'ния'],
 ['тре', 'рей', 'ейл', 'йле', 'лер', 'еро', 'ров'],
 ['и'],
 ['бол', 'оль', 'льш', 'ьше', 'шей'],
 ['инф', 'нфо', 'фор', 'орм', 'рма', 'мац', 'аци', 'ции'],
 ['пом', 'оме', 'ме

Взвешиваем векторы символов от первого до последнего - чтобы нейросеть понимала порядок

In [48]:
import numpy as np

def vectorize_ngram(ngram, vocab_dict, dim):
    return np.sum([vocab_dict.get(char, [0]*dim) / (1 + idx) for idx, char in enumerate(ngram.lower())], axis=0)

# Для каждой n-граммы считаем её вектор как сумму векторов символов
ngrams_vectors = []
for word_ngrams in ngrams:
    ngrams_vector = [vectorize_ngram(ngram, vocab_dict, dim) for ngram in word_ngrams]
    ngrams_vectors.append(ngrams_vector)

In [97]:
# Находим ближайших соседей n-граммы к вектору 
from sklearn.neighbors import NearestNeighbors

neigh = NearestNeighbors(n_neighbors=n_grams_len*2, metric="cosine")

# Фиттим сохраняя матчинг между символами и векторами
vectors = list(vocab_dict.values())
neigh.fit(vectors)

# Находим ближайших соседей
for ngrams_vector in ngrams_vectors[1]:
    vector1 = ngrams_vector
    distances, indices = neigh.kneighbors([vector1])

    # Выводим буквы и их ближайших соседей
    for index, distance in zip(indices[0][:n_grams_len], distances[0][:n_grams_len]):
        print(vocab[index], f"| {distance:.2f}")

    print("="*10)

ч | 0.16
е | 0.54
э | 0.73
е | 0.13
с | 0.60
т | 0.72
с | 0.16
н | 0.51
т | 0.63
т | 0.10
н | 0.52
о | 0.76


In [113]:
sample = vectorize_ngram('яеееее', vocab_dict, dim)
distances, indices = neigh.kneighbors([sample])

# Выводим буквы и их ближайших соседей
for index, distance in zip(indices[0], distances[0]):
    print(vocab[index], f"| {distance:.2f}")

print("="*10)

е | 0.16
я | 0.38
м | 0.80
x | 0.81
н | 0.81
щ | 0.85


In [75]:
import Levenshtein
import random

# Выбираем случайные n-граммы
# chosen_ngrams = random.sample([x for x in ngrams if len(x) > 2 and len(x) <= 3], 10)
# ngrams_flat = [ngram for ngrams in chosen_ngrams for ngram in ngrams]

ngrams_flat = ['мен', 'нем', 'емн', 'мне', 'сук', 'укс', 'кус']

# Составляем пары различных n-грамм
ngrams_pairs = []
for i, ngram in enumerate(ngrams_flat):
    for j, ngram2 in enumerate(ngrams_flat):
        if j > i:
            ngrams_pairs.append((ngram, ngram2))

# Считаем векторное расстояние между парами
cosine_distances = []
for ngram1, ngram2 in ngrams_pairs:
    vector1 = vectorize_ngram(ngram1, vocab_dict, dim)
    vector2 = vectorize_ngram(ngram2, vocab_dict, dim)
    cosine_distances.append(1 - np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))

# Считаем расстояния Левенштейна между теми же n-граммами
levenshtein_distances = []
for ngram1, ngram2 in ngrams_pairs:
    levenshtein_distances.append(Levenshtein.distance(ngram1, ngram2))

In [77]:
# Считаем корреляцию между векторными и Левенштейновскими расстояниями с p-value
from scipy.stats import pearsonr

correlation, p_value = pearsonr(cosine_distances, levenshtein_distances)
print(correlation, f"| p_value = {p_value:.2e}")

0.9714032453024829 | p_value = 2.53e-13


In [82]:
# принтим пары с их расстояниями
for ngrams, cosine_distance, levenshtein_distance in zip(ngrams_pairs, cosine_distances, levenshtein_distances):
    print(ngrams, f"| {cosine_distance:.2f} | {levenshtein_distance}")

('мен', 'нем') | 0.32 | 2
('мен', 'емн') | 0.13 | 2
('мен', 'мне') | 0.01 | 2
('мен', 'сук') | 0.81 | 3
('мен', 'укс') | 0.90 | 3
('мен', 'кус') | 0.85 | 3
('нем', 'емн') | 0.21 | 2
('нем', 'мне') | 0.26 | 2
('нем', 'сук') | 0.87 | 3
('нем', 'укс') | 1.02 | 3
('нем', 'кус') | 0.89 | 3
('емн', 'мне') | 0.18 | 2
('емн', 'сук') | 0.86 | 3
('емн', 'укс') | 0.89 | 3
('емн', 'кус') | 0.85 | 3
('мне', 'сук') | 0.81 | 3
('мне', 'укс') | 0.93 | 3
('мне', 'кус') | 0.86 | 3
('сук', 'укс') | 0.32 | 2
('сук', 'кус') | 0.37 | 2
('укс', 'кус') | 0.19 | 2
